In [1]:
pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 65.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556721 sha256=b2d6ee3390bd0aa1e33ecfda31252e0958cf4cffa615cb301710f2412988fbc8
  Stored in directory: /root/.cache/pip/wheels/a0/9e/13/a63ab80684c3bd6305fe2bded26ac5f2c72ee1cb07a747994d
Successfully built ipadic


In [2]:
pip install fugashi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 KB 36.9 MB/s eta 0:00:00


In [3]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.5/581.5 KB 37.9 MB/s eta 0:00:00


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.3 MB/s eta 0:00:00


In [5]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer

In [7]:
# df = pd.read_csv('sugakubunka.csv')
# df.dropna(inplace=True)
# df = df.drop(["Unnamed: 0","text_cleaned"],axis=1)
# df["label"] = df["label"].astype(int)
# df = df.replace({0:'NEGATIVE', 1:'POSITIVE'})
# df

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv('drive/My Drive/data/pn_cleaned.csv')
df["label"] = df["label"].astype(int)
df = df.replace({0:'NEGATIVE', 1:'POSITIVE'})
df

,label,text
0,NEGATIVE,ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…
1,POSITIVE,今日の月も白くて明るい。昨日より雲が少なくてキレイな〜 と立ち止まる帰り道。チャリなし生活も...
2,NEGATIVE,眠い、眠れない。
3,NEGATIVE,表情筋が衰えてきてる。まずいな…
4,NEGATIVE,おなかすいた…夜ご飯仲間募集。
...,...,...
23034,POSITIVE,久しぶりに珈琲淹れました。\nレシピ無しで自分の感覚を頼りに。\n\nそうすると、未来が拓け...
23035,POSITIVE,ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...
23036,POSITIVE,朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...
23037,POSITIVE,夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...


In [10]:
train_docs = df["text"].tolist()
train_labels = df["label"].tolist()

In [11]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [12]:
encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128)
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [13]:
optimizer = AdamW(model.parameters(), lr=1e-5)
batch_size = 200
num_batches = len(train_labels) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i+1) * batch_size

    batch_input_ids = input_ids[start_idx:end_idx]
    batch_attention_mask = attention_mask[start_idx:end_idx]
    batch_labels = [label2id[label] for label in train_labels[start_idx:end_idx]]
    batch_labels = torch.tensor(batch_labels).unsqueeze(0)

    outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114


In [14]:
# # Fine-tuning in native PyTorch

# # the AdamW() optimizer which implements gradient bias correction as well as weight decay.
# optimizer = AdamW(model.parameters(), lr=1e-5)
# labels = [label2id[label] for label in train_labels]
# labels = torch.tensor(labels).unsqueeze(0)
# outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
# loss = outputs.loss
# loss.backward()
# optimizer.step()

In [15]:
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
sentiment_analyzer("地震怖い")

[{'label': 'NEGATIVE', 'score': 0.7941070199012756}]

In [23]:
df_sugaku = pd.read_csv('drive/My Drive/data/sugakubunka.csv')
df_sugaku

,Unnamed: 0,text,text_cleaned,label
0,0,サンプルコード見ながらバグってる,"['サンプル', 'コード', '見る', 'バグ']",0.0
1,1,python多めの後にR触るとdata.trainとか書いてあるとき頭がバグる,"['多め', '後', '触る', 'DAT', 'A.T', 'RAIN', '書く', ...",0.0
2,2,高校の教員してた頃は数学の先生誰もTeX使ってなかったし勝手に使ってたらTeX禁止令出ちゃっ...,"['高校', '教員', '頃', '数学', '先生', '誰', 'TeX', '使う'...",NaN
3,4,昨日の講師勉強会、p進数とヘンゼルの補題の話をしてみました\n形を整えて講座にしたい気持ち😌...,"['昨日', '講師', '勉強会', '進数', 'HenzeL', '補題', '話',...",NaN
4,5,新しい名刺って作ってなかったっけ…\n古いのなら束であるんだけど…,"['新しい', '名刺', '作る', '古い', 'の', '束', 'ん']",NaN
...,...,...,...,...
172,195,肉眼ならいい感じに見えたのになー,"['肉眼', 'いい', '感じ', '見える']",1.0
173,196,都会の月食(うるとらわかりにくい) https://t.co/5wIeonJexx,"['都会', '月食', 'うる', 'とら', 'わかる', 'にくい']",0.0
174,197,復習します！,['復習'],1.0
175,198,講師勉強会、今日はとてもむじゅかしぃでした\n(ミュート外れてるのに気付かずに独り言聞かれた人),"['講師', '勉強会', '今日', '外れる', 'の', '気付く', '独り言', ...",1.0


In [44]:
df_sugaku['text'][9]

'はい！再来週やります！\n無料なのでお気軽にご参加ください😊 https://t.co/VfSozauXFf'

In [45]:
sentiment_analyzer(df_sugaku['text'][9])

[{'label': 'POSITIVE', 'score': 0.8955501317977905}]

In [48]:
dir_name = 'drive/My Drive/model'
sentiment_analyzer.save_pretrained(dir_name)

In [49]:
import pickle
filename = 'model.pkl'
pickle.dump(model,open(filename,'wb'))

In [60]:
saved_model = AutoModelForSequenceClassification.from_pretrained(dir_name)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
# model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
# tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [61]:
sentiment_analyzer_dl = pipeline("sentiment-analysis", model=saved_model, tokenizer=tokenizer)

In [62]:
sentiment_analyzer_dl(df_sugaku['text'][10])

[{'label': 'POSITIVE', 'score': 0.8954159617424011}]

In [63]:
sentiment_analyzer(df_sugaku['text'][10])

[{'label': 'POSITIVE', 'score': 0.8954159617424011}]